# Chapter 10. 시퀀스 해킹, 해시, 슬라이스
## 10.1 Vector: 사용자 정의 시퀀스형
- 요소들을 실수형 배열에 저장하고, 벡터가 불변 균일 시퀀스처럼 작동하게 만들기 위한 메서드들

## 10.2 Vector 버전 #1: Vector2d 호환
- 시퀀스 생성자는 내장 시퀀스처럼 반복형을 인수로 받게 만드는 것이 좋다.
- `reprlib`은 `repr()`에서 제한된 길이 제한을 할 수 있게 만들어준다. 


In [1]:
from vector_v1 import Vector
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

## 10.3 프로토콜과 덕 타이핑
- 시퀀스 프로토콜은 `__len__()`과 `__getitem()` 메서드만 동반하면
- 따로 시퀀스 프로토콜을 따른다고 명시하지 않아도 시퀀스처럼 동작


In [2]:
# 예제 10-3 [예제1-1]의 코드
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]


## 10.4 Vector 버전 #2: 슬라이스 가능한 시퀀스
~~~python
class Vector:
    ... 중략 ...
    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        return self._components[index]
~~~


In [3]:
from vector_v2 import Vector

v1 = Vector([3, 4, 5])
print(len(v1))
print(v1[0], v1[-1])

v7 = Vector(range(7))
v7[1:4]  # 결과물이 원하는대로 나오지 않는다.



3
3.0 5.0


Vector([1.0, 2.0, 3.0])

- `Vector`를 슬라이싱해서 생성된 배열은 자신과 다른 자료형 객체가 나오기 때문에 `Vector` 기능을 상실한다.

### 10.4.1 슬라이싱의 작동 방식


In [4]:
# 예제 10-4 __getitem__()과 slice()의 동작 방식
class MySeq:
    def __getitem__(self, index):
        return index
    
    
s = MySeq()
print(s[1])
print(s[1:4])
print(s[1:4:2])
print(s[1:4:2, 9])  # __getitem__()이 튜플을 받는다
print(s[1:4:2, 7:9])


1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


- `slice` 클래스에 `indices()` 메서드, `start`, `step`, `stop` 속성이 있다.
  - `S.indices(len)` 슬라이스 객체의 start, stop, step 속성을 길이 len짜리 시퀀스형에 적용될 수 있도록 (start, stop, stride)를 반환한다.

In [5]:
# 예제 10-5 slice 클래스의 속성조사
print(slice)
print(dir(slice))

<class 'slice'>
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'indices', 'start', 'step', 'stop']


In [6]:
print(slice(None, 10, 2).indices(5))
print(slice(-3, None, None).indices(5))


(0, 5, 2)
(2, 5, 1)


### 10.4.2 슬라이스를 인식하는 `__getitem__()`

~~~python
import numbers

def __getitem__(self, index):
    cls = type(self)
    if isinstance(index, slice):
        return cls(self._components[index])
    elif isinstance(index, numbers.Integral):
        return self._components[index]
    else:
        msg = f'{cls.__name} indices must be integers'
        raise TypeError(msg)
~~~

In [8]:
from vector_v2 import Vector

v1 = Vector([3, 4, 5])
print(len(v1))
print(v1[0], v1[-1])

v7 = Vector(range(7))
v7[1:4]  # 결과물이 원하는대로 나오지 않는다.


3
3.0 5.0


Vector([1.0, 2.0, 3.0])

## 10.5 Vector 버전 #3: 동적 속성 접근
- `인스턴스.속성` 구문은 먼저 속성이 있는지 검사하고 없으면, 클래스 속성이 있는지 검사한다. 상속 그래프를 따라 올라가도 속성을 못 찾으면 `__getattr__()` 메서드를 호출한다.

<br>

~~~python
shortcut_names = 'xyzt'

def __getattr__(self, name):
    cls = type(self)
    if len(name) == 1:
        pos = cls.shortcut_names.find(name)
        if 0 <= pos < len(self._components):
            return self._components[pos]
    raise AttributeError(f'{cls.__name__!r} object has no attribute {name!r}')
~~~

In [12]:
from vector_v3 import Vector
v = Vector(range(10))

v.x

v.x = 10
print(v.x)  # 인스턴스 속성 x가 할당되면서 더 이상 __getattr__() 메서드까지 가지 않는다.
print(v)

10
(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0)


- `__getattr__()` 메서드와 인스턴스 속성과의 불일치를 막기 위하여 `__setattr__()` 메서드를 구현

<br>

~~~python
def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
        if name in cls.shortcut_names:
            error = 'readonly attribute {attr_name!r}'
        elif name.islower():
            error = "can't set attributes 'a' to 'z' in {cls_name!r}"
        else:
            error = ''
        if error:
            msg = error.format(cls_name=cls.__name__, attr_name=name)
            raise AttributeError(msg)
    super().__setattr__(name, value)  # 에러가 발생하지 않으면 표준 동작을 위해 슈퍼클래스의 __setattr__() 메서드를 호출
~~~

In [1]:
from vector_v3 import Vector
v = Vector(range(10))

v.x

v.x = 10

AttributeError: readonly attribute 'x'

## 10.6 Vector 버전 #4: 해싱 및 더 빠른 ==
- Vector 안의 각 원소에 연속적으로 XOR 연산을 하기 위하여 `functools.reduce()`를 사용할 것이다.
- `reduce()`
  - 일련의 값을 하나의 값으로 줄이는 것
  - 첫 번째 인자: 두개의 인자를 받는 함수
  - 두 번째 인자: 반복형 객체
  - 세 번쨰 인자: (선택) 초기값. 빈 반복형 객체를 입력 받았을 때 에러가 발생하는 것을 방지. 항등원을 사용

In [3]:
import functools

functools.reduce(lambda a, b: a * b, range(1, 6))

120

In [5]:
# 예제 10-11 0에서 5까지 정수를 XOR로 누적 계산하는 세 가지 방법
import functools
import operator

n = 0
for i in range(6):
    n ^= i
print(n)

print(
    functools.reduce(lambda a, b: a ^ b, range(6))
)

print(
    functools.reduce(operator.xor, range(6))

)

1
1
1


~~~python
# 예제 10-12
import functools
import operator

class Vector:
    ...중략...
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)
~~~

<br>

제너레이터 표현식 대신 `map()`을 사용할 수 있다.

~~~python
import functools
import operator

class Vector:
    ...중략...
    def __hash__(self):
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)
~~~

<br>

<span style="color:green">느긋하게 수행한다는 의미가 무엇일까? => 제너레이터 Lazy하다. 미리 로드해놓는 것이 아니라 필요할 때 꺼내쓴다.</span>

<br>

고차원 벡터에 대해서도 빠르게 작동하는 `__eq__()` 메서드 구현

~~~python
# 예제 10-13 
class Vector:
    ...중략...
    def __eq__(self, other):
        if len(self) != len(other):  # zip은 짧은 쪽 반복형이 소진되면 멈추기 때문에 길이를 검사해야 한다.
            return False
        for a, b in zip(self, other):
            if a!= b:
                return False
        return True
~~~

<br>

~~~python
# 예제 10-14
class Vector:
    ...중략...
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
~~~

## 10.7 Vector 버전 #5: 포매팅
- 구면좌표계 구현

~~~python
class Vector:
    ...중략...
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi ** 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n for n in range(1, len(self))))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endwith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self), self.angles()])
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

~~~

In [7]:
from vector_v5 import Vector

print(format(Vector([-1, -1, -1, -1]), 'h'))
print(format(Vector([2, 2, 2, 2]), '.3eh'))
print(format(Vector([0, 1, 0, 0]), '0.5fh'))

<2.0, 2.0943951023931957, 2.186276035465284, 7.513409910897013>
<4.000e+00, 1.047e+00, 9.553e-01, 7.854e-01>
<1.00000, 1.57080, 0.00000, 0.00000>
